In [1]:
%load_ext rpy2.ipython

from fig_builder import TwinsFiguresData, TwinsFigures
import pysam
from plotnine import *
import pandas as pd
import warnings
import networkx as nx
from glob import glob
from os.path import isfile
from gimmebio.seqs import reverseComplement
import community

warnings.filterwarnings('ignore')

fig_data = TwinsFiguresData()
figs = TwinsFigures(fig_data)
SPARSE_N = 100

In [2]:
figs.metadata()

,date,pma_treated,kind,subject,flight,during_flight,time_label
011515_TW_B,2015-01-15,False,buccal,TW,before,before,before
011515_TW_S,2015-01-15,False,saliva,TW,before,before,before
012015_HR_B,2015-01-20,False,buccal,HR,ground,before,before
012015_HR_S,2015-01-20,False,saliva,HR,ground,before,before
012016_HR_B,2016-01-20,False,buccal,HR,ground,flight,flight
...,...,...,...,...,...,...,...
IIIF3SW,2016-05-06,False,Surface,ISS,NaN,NaN,unknown
IIIF8SW_P,2016-05-06,True,Surface,ISS,NaN,NaN,unknown
IIIF5SW_P,2016-05-06,True,Surface,ISS,NaN,NaN,unknown
IIIF3SW_P,2016-05-06,True,Surface,ISS,NaN,NaN,unknown


In [26]:
fname = 'strains/MHV-HR5_S41659407.cap2::experimental::align_to_genome.8c80eaf9bcec.genome_alignment__cutibacterium_acnes.bam'
bam = pysam.AlignmentFile(fname, 'rb')

for rec in bam:
    print(rec)
    break

NS500309:101:HLHLMBGXY:3:11606:17910:8009	99	5	8858	0	142M5I4M	5	8861	151	GATCAGGACGCCTACTCGACGATGAGTCGGGCCGACCTGGTGACGATGTTGGATTCGTTATCGGATTCGTCACGCCAGCTTGATTCTGAGATCGCTGACCTACGCGCTACGACAAGGGACCTCCAGTCGGGTGCTGATTCGCGTCCTTCCC	array('B', [32, 32, 32, 32, 32, 36, 36, 32, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 32, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 32, 36, 36, 32, 32, 36, 36, 36, 36, 36, 36, 36, 36, 32, 21])	[('AS', -34), ('XS', -34), ('XN', 0), ('XM', 3), ('XO', 1), ('XG', 5), ('NM', 8), ('MD', '142G1A0A0'), ('YS', -38), ('YT', 'CP')]


In [3]:
rec.get_forward_sequence()

'GATCAGGACGCCTACTCGACGATGAGTCGGGCCGACCTGGTGACGATGTTGGATTCGTTATCGGATTCGTCACGCCAGCTTGATTCTGAGATCGCTGACCTACGCGCTACGACAAGGGACCTCCAGTCGGGTGCTGATTCGCGTCCTTCCC'

In [4]:
rec.get_reference_sequence()

'GATCAGGACGCCTACTCGACGATGAGTCGGGCCGACCTGGTGACGATGTTGGATTCGTTATCGGATTCGTCACGCCAGCTTGATTCTGAGATCGCTGACCTACGCGCTACGACAAGGGACCTCCAGTCGGGTGCTGATTCGCgCaa'

In [5]:
rec.reference_name

'NZ_VBYB01000014.1'

In [6]:
rec.reference_start

8858

In [27]:
def get_mismatches(rec):
    qseq = rec.get_forward_sequence().upper()
    if rec.is_reverse:
        qseq = reverseComplement(qseq)
    rseq = rec.get_reference_sequence().upper()
    for qpos, rpos in rec.get_aligned_pairs():
        if qpos == None or rpos == None:
            continue  # no indels yet
        q = qseq[qpos]
        r = rseq[rpos - rec.reference_start]
        if q != r:
            position = (rec.reference_name, rpos)
            change = (r, q)
            yield (position, change)
            

def build_graph(rec_iter, G=nx.Graph()):
    for rec in rec_iter:
        misses = list(get_mismatches(rec))
        for missA in misses:
            for missB in misses:
                if missA == missB:
                    break
                try:
                    w = G[missA][missB]['weight']
                except KeyError:
                    w = 0
                G.add_edge(missA, missB, weight=w + 1)
    return G
    

G = build_graph(bam)
print(G.number_of_nodes())
print(G.number_of_edges())

G

13729
163852


In [20]:
filtered_graph = nx.Graph()

for u, v, data in G.edges(data=True):
    if data['weight'] >= 2:
        filtered_graph.add_edge(u, v)

for subg in nx.connected_component_subgraphs(filtered_graph):
    print(subg.number_of_nodes())

In [28]:
partition = community.best_partition(G)

sizes = {}
for _, c in partition.items():
    sizes[c] = sizes.get(c, 0) + 1

print('mean', sum(sizes.values()) / len(sizes))
print('median', sorted(sizes.values())[len(sizes) // 2])

mean 27.735353535353536
median 18


In [40]:
weights = {}

for node in G.nodes():
    total_weight = 0
    for el in G.edges((('NZ_VBYB01000014.1', 9003), ('A', 'C')), data=True):
        total_weight += el[2]['weight']
    weights[node] = total_weight
    
weights

{(('NZ_VBYB01000014.1', 9003), ('A', 'C')): 2,
 (('NZ_VBYB01000014.1', 9002), ('A', 'C')): 2,
 (('NZ_VBYB01000014.1', 9004), ('G', 'C')): 2,
 (('NZ_VBYB01000001.1', 95852), ('A', 'C')): 2,
 (('NZ_VBYB01000001.1', 95850), ('A', 'C')): 2,
 (('NZ_VBYB01000001.1', 95855), ('T', 'C')): 2,
 (('NZ_VBYB01000001.1', 95862), ('G', 'A')): 2,
 (('NZ_VBYB01000001.1', 95865), ('T', 'C')): 2,
 (('NZ_VBYB01000001.1', 95869), ('T', 'G')): 2,
 (('NZ_VBYB01000001.1', 95883), ('A', 'G')): 2,
 (('NZ_VBYB01000001.1', 95890), ('C', 'T')): 2,
 (('NZ_VBYB01000001.1', 95892), ('T', 'C')): 2,
 (('NZ_VBYB01000001.1', 95899), ('G', 'A')): 2,
 (('NZ_VBYB01000001.1', 95900), ('A', 'T')): 2,
 (('NZ_VBYB01000001.1', 95901), ('T', 'A')): 2,
 (('NZ_VBYB01000001.1', 95906), ('C', 'A')): 2,
 (('NZ_VBYB01000001.1', 95910), ('G', 'C')): 2,
 (('NZ_VBYB01000001.1', 95912), ('G', 'A')): 2,
 (('NZ_VBYB01000001.1', 95914), ('A', 'G')): 2,
 (('NZ_VBYB01000001.1', 95915), ('G', 'C')): 2,
 (('NZ_VBYB01000001.1', 95963), ('G', 'A'))